# Catboost, XGBoost and LightGBM

## Learning Objectives
At the end of the experiment, you will be able to :

* perform data preprocessing
* perform feature transformation
* implement CatBoost, XGBoost and LightGBM model to perform classification using Lending Club dataset

## Introduction

**XGBoost** was originally produced by University of Washington researchers and is maintained by open-source contributors. XGBoost is available in Python, R, Java, Ruby, Swift, Julia, C, and C++. Similar to LightGBM, XGBoost uses the gradients of different cuts to select the next cut, but XGBoost also uses the hessian, or second derivative, in its ranking of cuts. Computing this next derivative comes at a slight cost, but it also allows a greater estimation of the cut to use.

**CatBoost** is developed and maintained by the Russian search engine Yandex and is available in Python, R, C++, Java, and also Rust. CatBoost distinguishes itself from LightGBM and XGBoost by focusing on optimizing decision trees for categorical variables, or variables whose different values may have no relation with each other (eg. apples and oranges).

**LightGBM** is a boosting technique and framework developed by Microsoft. The framework implements the LightGBM algorithm and is available in Python, R, and C. LightGBM is unique in that it can construct trees using Gradient-Based One-Sided Sampling, or GOSS for short.

To know more on comparisons between CatBoost, XgBoost and LightGBM, refer below
- [Article 1](https://towardsdatascience.com/catboost-vs-light-gbm-vs-xgboost-5f93620723db)
- [Article 2](https://towardsdatascience.com/catboost-vs-lightgbm-vs-xgboost-c80f40662924)

## Dataset Description

Lending Club is a lending platform that lends money to people in need at an interest rate based on their credit history and other factors. We will analyze this data and pre-process it based on our need and build a machine learning model that can identify a potential defaulter based on his/her history of transactions with Lending Club. 

This dataset contains 42538 rows and 144 columns. **Out of these 144 columns, many columns have majorly null values.**

To know more about the Lending Club dataset features, refer [here](https://www.openintro.org/data/index.php?data=loans_full_schema).

###  Import required packages

In [ ]:
!pip -qq install catboost

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier, Pool, metrics, cv
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')

### Load Dataset

In [ ]:
!wget https://github.com/Adithya-Thonse/python_ML_DL_basics/upload/main/ML/Datasets/LoanStats3a.zip

In [ ]:
import zipfile
with zipfile.ZipFile("LoanStats3a.zip","r") as zip_ref:
    zip_ref.extractall("./")

In [ ]:
# Load the raw loan stats dataset
data = pd.read_csv("LoanStats3a.csv")
data.shape

## Data Preprocessing

In [ ]:
# View the top 5 rows of data
pd.set_option('display.max_columns', None)

data.head(5)

In [ ]:
# Size of the dataset
data.shape

In [ ]:
# Checking info of the raw dataframe
data.info()

### Check for missing values in the dataset

In [ ]:
# Check missing values
data.isnull().sum()

In [ ]:
# Total percentage of null values in the data
pct = (data.isnull().sum().sum())/(data.shape[0]*data.shape[1])
print("Overall missing values in the data ≈ {:.2f} %".format(pct*100))

From above we can see that, about 63% of the values in the overall data are null values.

Let's visualize the null values using the heatmap.

In [ ]:
# Checking for null values using a heatmap as a visualizing tool
plt.figure(figsize=(16,14))
sns.heatmap(data.isnull())
plt.title('Null values heat plot', fontdict={'fontsize': 20})
plt.legend(data.isnull())
plt.show()

As we can see from the above heatmap, there are lot of null values in the dataset. We have to carefully deal with these null values.

### Handling missing values in the data

- Select columns having null values less than 40%

In [ ]:
# Creating a dataframe to display percentage of null values in different number of columns
temp_df = pd.DataFrame()
temp_df['Percentage of null values'] = ['10% or less', '10% to 20%', '20% to 30%', '30% to 40%', '40% to 50%', 
                                        '50% to 60%', '60% to 70%', '70% to 80%', '80% to 90%', 'More than 90%']

# Store the columns count separately for each range
ten_percent = len(data.columns[((data.isnull().sum())/len(data)) <= 0.1])
ten_to_twenty_percent = len(data.columns[((data.isnull().sum())/len(data)) <= 0.2] & data.columns[((data.isnull().sum())/len(data)) > 0.1])
twenty_to_thirty_percent = len(data.columns[((data.isnull().sum())/len(data)) <= 0.3] & data.columns[((data.isnull().sum())/len(data)) > 0.2])
thirty_to_forty_percent = len(data.columns[((data.isnull().sum())/len(data)) <= 0.4] & data.columns[((data.isnull().sum())/len(data)) > 0.3])
forty_to_fifty_percent = len(data.columns[((data.isnull().sum())/len(data)) <= 0.5] & data.columns[((data.isnull().sum())/len(data)) > 0.4])
fifty_to_sixty_percent = len(data.columns[((data.isnull().sum())/len(data)) <= 0.6] & data.columns[((data.isnull().sum())/len(data)) > 0.5])
sixty_to_seventy_percent = len(data.columns[((data.isnull().sum())/len(data)) <= 0.7] & data.columns[((data.isnull().sum())/len(data)) > 0.6])
seventy_to_eighty_percent = len(data.columns[((data.isnull().sum())/len(data)) <= 0.8] & data.columns[((data.isnull().sum())/len(data)) > 0.7])
eighty_to_ninety_percent = len(data.columns[((data.isnull().sum())/len(data)) <= 0.9] & data.columns[((data.isnull().sum())/len(data)) > 0.8])
hundred_percent = len(data.columns[((data.isnull().sum())/len(data)) > 0.9])

temp_df['No. of columns'] = [ten_percent, ten_to_twenty_percent, twenty_to_thirty_percent, thirty_to_forty_percent, forty_to_fifty_percent, 
                             fifty_to_sixty_percent, sixty_to_seventy_percent, seventy_to_eighty_percent, eighty_to_ninety_percent, hundred_percent]
temp_df

From the above results, we can see that there are only 53 columns out of 144 columns that have null values less than 40%.

In [ ]:
# Considering only those columns which have null values less than 40% in that particular column
df1 = data[data.columns[((data.isnull().sum())/len(data)) < 0.4]]
df1.shape

By considering columns with less number of null values, we were able to decrease total number of columns from 144 to 53.

Note that we will deal with null values present in these selected 53 columns later below.

### Removing columns having single distinct value

In [ ]:
# Checking columns that have only single values in them i.e, constant columns
const_cols = []
for i in df1.columns:
    if df1[i].nunique() == 1:
        const_cols.append(i)

print(const_cols)

In [ ]:
# After observing the above output, we are dropping columns which have single values in them
print("Shape before:", df1.shape)
df1.drop(const_cols, axis=1, inplace = True)
print("Shape after:", df1.shape)

### Extract features from datetime columns

In [ ]:
# Columns other than numerical value
colms = df1.columns[df1.dtypes == 'object']
colms

In [ ]:
# Check which columns needs to be converted to datetime
df1[colms].head(2)

In [ ]:
# Converting objects to datetime columns
dt_cols = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']
for i in dt_cols:
    df1[i] = pd.to_datetime(df1[i].astype('str'), format='%b-%y', yearfirst=False)

In [ ]:
# Checking the new datetime columns
df1[['issue_d','earliest_cr_line','last_pymnt_d','last_credit_pull_d']].head()

In [ ]:
# Considering only year of joining for 'earliest_cr_line' column
df1['earliest_cr_line'] = pd.DatetimeIndex(df1['earliest_cr_line']).year

In [ ]:
# Adding new features by getting month and year from [issue_d, last_pymnt_d, and last_credit_pull_d] columns
df1['issue_d_year'] = pd.DatetimeIndex(df1['issue_d']).year  
df1['issue_d_month'] = pd.DatetimeIndex(df1['issue_d']).month  
df1['last_pymnt_d_year'] = pd.DatetimeIndex(df1['last_pymnt_d']).year  
df1['last_pymnt_d_month'] = pd.DatetimeIndex(df1['last_pymnt_d']).month 
df1['last_credit_pull_d_year'] = pd.DatetimeIndex(df1['last_credit_pull_d']).year  
df1['last_credit_pull_d_month'] = pd.DatetimeIndex(df1['last_credit_pull_d']).month 

In [ ]:
# Feature extraction
df1.earliest_cr_line = 2019 - (df1.earliest_cr_line)
df1.issue_d_year = 2019 - (df1.issue_d_year)
df1.last_pymnt_d_year = 2019 - (df1.last_pymnt_d_year)
df1.last_credit_pull_d_year = 2019 - (df1.last_credit_pull_d_year)

In [ ]:
# Dropping the original features to avoid data redundancy
df1.drop(['issue_d','last_pymnt_d','last_credit_pull_d'], axis=1, inplace=True)
df1.shape

### Check for missing values in reduced dataset

In [ ]:
# Checking for null values in the updated dataframe
plt.figure(figsize=(16,10))
sns.heatmap(df1.isnull())
plt.show()

### Handling Null values in reduced dataset

In [ ]:
# Checking for Percentage of null values
a = (df1.isnull().sum() / df1.shape[0]) * 100
b = a[a > 0.00]
b = pd.DataFrame(b, columns = ['Percentage of null values'])
b.sort_values(by= ['Percentage of null values'], ascending=False)

In [ ]:
# Dropping the 29 rows which have null values in few columns
df1 = df1[df1['delinq_2yrs'].notnull()]
df1.shape

In [ ]:
# Checking again for Percentage of null values
a = (df1.isnull().sum() / df1.shape[0]) * 100
b = a[a > 0.00]
b = pd.DataFrame(b, columns = ['Percentage of null values'])
b.sort_values(by= ['Percentage of null values'], ascending=False)

Now, imputing the missing values with the median value for columns **'last_pymnt_d_year', 'last_pymnt_d_month', 'last_credit_pull_d_year', 'last_credit_pull_d_month', 'tax_liens'** as null values in these columns are less than 0.5% of the size.

In [ ]:
# Imputing the null values with the median value
df1['last_pymnt_d_year'].fillna(df1['last_pymnt_d_year'].median(), inplace=True)
df1['last_pymnt_d_month'].fillna(df1['last_pymnt_d_month'].median(), inplace=True)
df1['last_credit_pull_d_year'].fillna(df1['last_credit_pull_d_year'].median(), inplace=True)
df1['last_credit_pull_d_month'].fillna(df1['last_credit_pull_d_month'].median(), inplace=True)
# df1['tax_liens'].fillna(df1['tax_liens'].median(), inplace=True)

For **'revol_util'** column, filling null values with median(string) which is close to 50:

In [ ]:
# For 'revol_util' column, fill null values with 50%
df1.revol_util.fillna('50%', inplace=True)  

# Extracting numerical value from string
df1.revol_util = df1.revol_util.apply(lambda x: x[:-1])

# Converting string to float
df1.revol_util = df1.revol_util.astype('float')

In [ ]:
# Unique values in 'pub_rec_bankruptcies' column
df1.pub_rec_bankruptcies.value_counts()

From the above we can see that the **'pub_rec_bankruptcies'** column is highly imbalanced. So, it is better to fill it with median(0) value as even after building model the model will be skewed very much towards 0.

In [ ]:
# Fill 'pub_rec_bankruptcies' column
df1['pub_rec_bankruptcies'].fillna(df1['pub_rec_bankruptcies'].median(), inplace=True)

In [ ]:
# Unique values in 'emp_length' column
df1['emp_length'].value_counts()

In [ ]:
# Seperating null values by assigning a random string
df1['emp_length'].fillna('5000',inplace=True) 

# Filling '< 1 year' as '0 years' of experience and '10+ years' as '10 years'
df1.emp_length.replace({'10+ years':'10 years', '< 1 year':'0 years'}, inplace=True)

# Then extract numerical value from the string
df1.emp_length = df1.emp_length.apply(lambda x: x[:2])

# Converting it's dattype to float
df1.emp_length = df1.emp_length.astype('float')

In [ ]:
# Checking again for Percentage of null values
a = (df1.isnull().sum() / df1.shape[0]) * 100
b = a[a > 0.00]
b = pd.DataFrame(b, columns = ['Percentage of null values'])
b.sort_values(by= ['Percentage of null values'], ascending=False)

In [ ]:
# Removing redundant features and features which have percentage null values > 5%
df1.drop(['desc', 'emp_title', 'title'], axis = 1, inplace = True)
df1.isnull().sum()

### Converting categorical columns to numerical columns


In [ ]:
df1.head(2)

In [ ]:
# Unique values in 'term' column
df1['term'].unique()

In [ ]:
# Unique values in 'int_rate' column
df1['int_rate'].unique()[:5]

In [ ]:
# Converting 'term' and 'int_rate' to numerical columns
df1.term = df1.term.apply(lambda x: x[1:3])
df1.term = df1.term.astype('float')
df1.int_rate = df1.int_rate.apply(lambda x: x[:2])
df1.int_rate = df1.int_rate.astype('float')
df1.head(2)

Among the address related features, considering **'addr_state'** column and excluding **'zip_code'** column.

In [ ]:
df2 = df1.drop('zip_code', axis = 1)

In [ ]:
# One hot encoding on categorical columns
df2 = pd.get_dummies(df2, columns = ['home_ownership', 'verification_status', 'purpose', 'addr_state', 'debt_settlement_flag'], drop_first = True)
df2.head(2)

In [ ]:
# Label encoding on 'grade' column
le = LabelEncoder()
le.fit(df2.grade)
print(le.classes_)

In [ ]:
# Update 'grade' column
df2.grade = le.transform(df2.grade)

In [ ]:
# Label encoding on 'sub_grade' column
le2 = LabelEncoder()
le2.fit(df2.sub_grade)
le2.classes_

In [ ]:
# Update 'sub_grade' column
df2.sub_grade = le2.transform(df2.sub_grade)

In [ ]:
df2.head(2)

In [ ]:
# Target feature
df2['loan_status'].unique()

In [ ]:
# Prediction features
X = df2.drop("loan_status", axis = 1)
# Target variable
y = df2['loan_status'] 
y.value_counts()

In [ ]:
# Label encoding the target variable
le3 = LabelEncoder()
le3.fit(y)
y_transformed = le3.transform(y)
y_transformed

In [ ]:
X.head(2)

### Split data into training and testing set

In [ ]:
# Split the data into train and test
x_train, x_test, y_train, y_test = train_test_split(X, y_transformed, test_size = 0.20, stratify = y_transformed, random_state = 2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

## Model Building

In [ ]:
# Using DecisionTree as base model
giniDecisionTree = DecisionTreeClassifier(criterion='gini', random_state = 100, 
                                          max_depth=3, class_weight = 'balanced', min_samples_leaf = 5)
giniDecisionTree.fit(x_train, y_train)

In [ ]:
# Prediciton using DecisionTree
giniPred = giniDecisionTree.predict(x_test)
print('Accuracy Score: ', accuracy_score(y_test, giniPred))

### CatBoost

In [ ]:
# Create CatBoostClassifier object
CatBoost_clf = CatBoostClassifier(iterations=5,
                                  learning_rate=0.1,
                                  #loss_function='CrossEntropy'
                                  )

In [ ]:
#cat_features = list(range(0, X.shape[1]))
CatBoost_clf.fit(x_train, y_train,
                 #cat_features=cat_features,
                 eval_set = (x_test, y_test),
                 verbose = False)

In [ ]:
# Prediction using CatBoost
cbr_prediction = CatBoost_clf.predict(x_test)
print('Accuracy Score: ', accuracy_score(y_test, cbr_prediction))

In [ ]:
#  Classification report for CatBoost model
print('Classification Report for CatBoost:')
print(classification_report(y_test, cbr_prediction))

### XGBoost

In [ ]:
# Create XGBClassifier object
XGB_clf = XGBClassifier(learning_rate = 0.1)

In [ ]:
# Fit on training set
XGB_clf.fit(x_train, y_train,
            eval_set = [(x_train, y_train), (x_test, y_test)],
            verbose = False)

In [ ]:
# Prediction using XGBClassifier
XGB_prediction = XGB_clf.predict(x_test)
print('Accuracy Score: ', accuracy_score(y_test, XGB_prediction))

In [ ]:
# Classification report for XGBoost

print('Classification Report for XGBoost:')
print(classification_report(y_test, XGB_prediction))

### LightGBM

In [ ]:
# Create LGBMClassifier object
LGBM_clf = LGBMClassifier(learning_rate = 0.1)

In [ ]:
# Fit on training set
LGBM_clf.fit(x_train, y_train,
             eval_set = [(x_train, y_train), (x_test, y_test)],
             verbose = False)

In [ ]:
# Prediction using LGBMClassifier
LGBM_prediction = LGBM_clf.predict(x_test)
print('Accuracy Score: ', accuracy_score(y_test, LGBM_prediction))

In [ ]:
# Classification report for LGBM

print('Classification Report for LGBM:')
print(classification_report(y_test, LGBM_prediction))

## Reference Reading:

https://neptune.ai/blog/when-to-choose-catboost-over-xgboost-or-lightgbm